# Task 1

<img src="ex1.png" alt="Task 1 pic" width="550" style="float:left"/>

In [1]:
SQL_QUERY = '''
--creating row numbers
WITH matrices_numerate AS (
  SELECT ROW_NUMBER() OVER(ORDER BY matrix) AS rn,* 
  FROM public.matrices
)
  
--aggregation of transposed rows into transpose matrices
SELECT rn, ARRAY_AGG(new_rows)
FROM (
  --generating transposed rows with new column indices
  SELECT rn, j, ARRAY_AGG(matrix[i][j]) AS new_rows
  FROM matrices_numerate AS m
  CROSS JOIN generate_subscripts(m.matrix, 1) AS i
  CROSS JOIN generate_subscripts(m.matrix, 2) AS j
  GROUP BY rn, j
  ORDER BY rn ASC, j ASC
) t
GROUP BY rn
'''

## [Query result](https://dbfiddle.uk/C9ZOhIei):

<img src="query_result.png" alt="Query" width="700" style="float:left"/>

## More detailed explanation of the sql-request with comments [here](https://dbfiddle.uk/OSFO1SP5)

# Task 2

<img src="ex2.png" alt="Task 2 pic" width="500" style="float:left"/>

<img src="df_picture.png" alt="Task 2 pic" width="900" style="float:left"/>

Шаги:
1. Проверить наблюдения с отсутствующими значениями
2. Убрать признаки с большим количеством NA
3. Проверить, что данные собраны нормально (выбросы)
4. Заменить выбросы/аномалии на среднее, медиану (для количественных значений) или моду (для категориальных) если потребуется
5. Выбросить неинформативные/нерелевантные признаки или же дубликаты

In [2]:
import pandas as pd
import warnings

from pandas_profiling import ProfileReport

warnings.filterwarnings("ignore")

In [3]:
df = pd.read_csv('data/ifood_df.csv')

Отсортирую колонки для удобства

In [4]:
col_sorted = list(df.columns)
col_sorted.sort()

In [5]:
df_sorted_col = df[col_sorted]

Посмотрю, есть ли отсутствующие значения и формат самих значений

In [6]:
df_sorted_col.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2205 entries, 0 to 2204
Data columns (total 39 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   AcceptedCmp1          2205 non-null   int64  
 1   AcceptedCmp2          2205 non-null   int64  
 2   AcceptedCmp3          2205 non-null   int64  
 3   AcceptedCmp4          2205 non-null   int64  
 4   AcceptedCmp5          2205 non-null   int64  
 5   AcceptedCmpOverall    2205 non-null   int64  
 6   Age                   2205 non-null   int64  
 7   Complain              2205 non-null   int64  
 8   Customer_Days         2205 non-null   int64  
 9   Income                2205 non-null   float64
 10  Kidhome               2205 non-null   int64  
 11  MntFishProducts       2205 non-null   int64  
 12  MntFruits             2205 non-null   int64  
 13  MntGoldProds          2205 non-null   int64  
 14  MntMeatProducts       2205 non-null   int64  
 15  MntRegularProds      

NaN'ов нет

Посмотрю детальнее на данные колонок

In [7]:
df_sorted_col.describe().T

,count,mean,std,min,25%,50%,75%,max
AcceptedCmp1,2205.0,0.064399,0.245518,0.0,0.0,0.0,0.0,1.0
AcceptedCmp2,2205.0,0.013605,0.115872,0.0,0.0,0.0,0.0,1.0
AcceptedCmp3,2205.0,0.073923,0.261705,0.0,0.0,0.0,0.0,1.0
AcceptedCmp4,2205.0,0.074376,0.262442,0.0,0.0,0.0,0.0,1.0
AcceptedCmp5,2205.0,0.073016,0.260222,0.0,0.0,0.0,0.0,1.0
AcceptedCmpOverall,2205.0,0.299320,0.680440,0.0,0.0,0.0,0.0,4.0
Age,2205.0,51.095692,11.705801,24.0,43.0,50.0,61.0,80.0
Complain,2205.0,0.009070,0.094827,0.0,0.0,0.0,0.0,1.0
Customer_Days,2205.0,2512.718367,202.563647,2159.0,2339.0,2515.0,2688.0,2858.0
Income,2205.0,51622.094785,20713.063826,1730.0,35196.0,51287.0,68281.0,113734.0


## Значение фичей:

<img src="data/dictionary.png" alt="dictionary" width="800" style="float:left">

Данные на картинке и те, которые есть в датафрейме немного отличаются - подведу итоги

* Всего 6 видов продуктов: 
    * Рыба
    * Мясо
    * Фрукты
    * Сладости
    * Вина


* Которые в свою очередь делятся по премиальности: 
    * regular-продукты
    * gold-продукты


* Продаются по 4 каналам: 
    * магазины
    * по дисконту
    * каталоги
    * вебсайт


Какие есть данные по покупателям:
* Принял ли офер какой-либо из рекламных кампаний
* Возраст
* Жаловался ли за последние 2 года
* Сколько дней назад клиент зарегистрировался в компании?
* Траты в разрезе типов продуктов
* Траты в разрезе премиальности 
* Количество покупок в разрезе по каналу продажи
* Дней с прошлой покупки
* Посещений вебсайта за месяц
* Дней с последней покупки
* Сколько детей-подростков
* Образование
* Семейный статус

Ещё есть колонки без описания: **Z_CostContact** и **Z_revenue** которые не несут дополнительной информации, так как имеют одинаковое значение во всех строках — в дальнейшем не буду их использовать

In [8]:
df_sorted_col.columns

Index(['AcceptedCmp1', 'AcceptedCmp2', 'AcceptedCmp3', 'AcceptedCmp4',
       'AcceptedCmp5', 'AcceptedCmpOverall', 'Age', 'Complain',
       'Customer_Days', 'Income', 'Kidhome', 'MntFishProducts', 'MntFruits',
       'MntGoldProds', 'MntMeatProducts', 'MntRegularProds',
       'MntSweetProducts', 'MntTotal', 'MntWines', 'NumCatalogPurchases',
       'NumDealsPurchases', 'NumStorePurchases', 'NumWebPurchases',
       'NumWebVisitsMonth', 'Recency', 'Response', 'Teenhome', 'Z_CostContact',
       'Z_Revenue', 'education_2n Cycle', 'education_Basic',
       'education_Graduation', 'education_Master', 'education_PhD',
       'marital_Divorced', 'marital_Married', 'marital_Single',
       'marital_Together', 'marital_Widow'],
      dtype='object')

Ну и сразу как по описанию фичей, так и по describe() видно, что есть категориальные значения, над которыми сделали onehotencoding - Response, Complain. Возможно, и AcceptedCmp, но, скорее всего должны быть строки, где сразу в нескольких кампаниях стоит "1" - проверю сразу

In [9]:
df_sorted_col[(df_sorted_col['AcceptedCmp1'] == 1) & (df_sorted_col['AcceptedCmp3'] == 1)].shape

(24, 39)

In [10]:
df_sorted_col[(df_sorted_col['AcceptedCmp1'] == 1) & (df_sorted_col['AcceptedCmp2'] == 1)].sample()

,AcceptedCmp1,AcceptedCmp2,AcceptedCmp3,AcceptedCmp4,AcceptedCmp5,AcceptedCmpOverall,Age,Complain,Customer_Days,Income,...,education_2n Cycle,education_Basic,education_Graduation,education_Master,education_PhD,marital_Divorced,marital_Married,marital_Single,marital_Together,marital_Widow
1896,1,1,0,1,1,4,64,0,2776,91249.0,...,0,0,1,0,0,0,0,1,0,0


Так и есть

Пробегусь ещё глазами по случайным сэмплам таблицы

In [11]:
df_sorted_col.sample(5).T

,1537,806,1180,16,584
AcceptedCmp1,0.0,0.0,0.0,0.0,0.0
AcceptedCmp2,0.0,0.0,0.0,0.0,0.0
AcceptedCmp3,0.0,0.0,0.0,0.0,0.0
AcceptedCmp4,0.0,1.0,0.0,0.0,0.0
AcceptedCmp5,0.0,1.0,0.0,0.0,0.0
AcceptedCmpOverall,0.0,2.0,0.0,0.0,0.0
Age,32.0,66.0,43.0,74.0,60.0
Complain,0.0,0.0,0.0,0.0,0.0
Customer_Days,2764.0,2577.0,2450.0,2826.0,2826.0
Income,24645.0,93404.0,54809.0,37760.0,62807.0


Проверю на выбросы

In [12]:
def outliers(dataframe: pd.DataFrame, columns: str) -> dict[str, int]:
    out_dict = {}
    for col in columns:
        q1 = dataframe[col].quantile(0.25)
        q3 = dataframe[col].quantile(0.75)
        iqr = q3 - q1
        
        outliers = dataframe[(dataframe[col] > (q3 + 1.5*iqr)) | (dataframe[col] < (q1 - 1.5*iqr))]
        out_dict[col] = outliers.shape[0]
        print(outliers[[col]].sort_values(by=col, ascending=False))
    return out_dict

In [13]:
mnt_cols = ['MntWines', 'MntFruits', 'MntMeatProducts', 
            'MntFishProducts', 'MntSweetProducts', 'MntGoldProds', 
            'NumDealsPurchases', 'NumWebPurchases', 'NumCatalogPurchases', 
            'NumStorePurchases', 'NumWebVisitsMonth']

In [14]:
outliers(df, mnt_cols)

      MntWines
1723      1493
542       1492
804       1492
2066      1486
526       1478
897       1478
1935      1462
1616      1459
1463      1449
480       1396
967       1394
506       1379
150       1349
102       1332
1896      1324
1032      1315
2095      1311
850       1308
1433      1302
1171      1298
970       1296
498       1288
981       1285
1927      1285
407       1285
806       1279
1966      1276
1467      1259
917       1253
1552      1252
413       1248
2039      1245
281       1241
449       1239
      MntFruits
1455        199
835         199
1483        197
100         194
1501        194
...         ...
685          80
2036         80
1139         80
909          80
334          80

[245 rows x 1 columns]
      MntMeatProducts
20               1725
1337              984
1782              981
439               974
2077              968
...               ...
274               565
298               561
1503              560
669               559
1697             

{'MntWines': 34,
 'MntFruits': 245,
 'MntMeatProducts': 170,
 'MntFishProducts': 222,
 'MntSweetProducts': 238,
 'MntGoldProds': 204,
 'NumDealsPurchases': 82,
 'NumWebPurchases': 3,
 'NumCatalogPurchases': 20,
 'NumStorePurchases': 0,
 'NumWebVisitsMonth': 8}

Не сказал бы, что они мега существенные и нужно с ними что-то делать - оставляю как есть

Посмотрю более детально через pandas-profiling

In [15]:
#profile = ProfileReport(df_sorted_col)
#profile

In [16]:
df_sorted_col[df_sorted_col['MntRegularProds'] < 0].T

,1307,1780,1949
AcceptedCmp1,0.0,0.0,0.0
AcceptedCmp2,0.0,0.0,0.0
AcceptedCmp3,0.0,0.0,0.0
AcceptedCmp4,0.0,0.0,0.0
AcceptedCmp5,0.0,0.0,0.0
AcceptedCmpOverall,0.0,0.0,0.0
Age,38.0,54.0,51.0
Complain,0.0,0.0,0.0
Customer_Days,2358.0,2363.0,2426.0
Income,6560.0,7144.0,4428.0


Датасет довольно чистенький, пропущенных значений нет. Без серьёзных выбросов. Из аномалий - несколько отрицательных значений MntRegularProds(возможно, вернули товар). Сделаю обратный OneHotEncoding для дальнейшего удобства и, может, для некоторых фичей Total через сумму. В дальнейшей обработке, помимо сказанного и сделанного нужды не вижу. 


Визуализации сделаю уже в Tableau

Идеи:
* OneHotEncoding reverse для дальнейшего удобства визуализаций в Tableau
* ChildrenTotal как сумма Kidhome и Teenager

Tableau:
* Общие распределения
* Посмотреть распределение мер в разрезе
    * Типа продуктов
    * Премиальности
    * Канала продажи
    * Образования
    * Количества детей
* Корреляции между мерами

Reverse OHE:

In [17]:
marital_cols = [col for col in df.columns if('marital' in col)]
ed_cols = [col for col in df.columns if('education' in col)]

In [18]:
#Реверс OHE по нужным колонкам + убираю префиксы Marital_, Education_
marital = df_sorted_col[marital_cols].idxmax(axis=1).\
    apply(lambda x: x.split('_')[1])
education = df_sorted_col[ed_cols].idxmax(axis=1).\
    apply(lambda x: x.split('_')[1])

In [19]:
df_sorted_col['Marital'] = marital
df_sorted_col['Education'] = education

In [20]:
df_sorted_col['Marital']

0         Single
1         Single
2       Together
3       Together
4        Married
          ...   
2200     Married
2201    Together
2202    Divorced
2203    Together
2204     Married
Name: Marital, Length: 2205, dtype: object

In [21]:
df_sorted_col[['Marital', 'marital_Single', 'marital_Together']].head()

,Marital,marital_Single,marital_Together
0,Single,1,0
1,Single,1,0
2,Together,0,1
3,Together,0,1
4,Married,0,0


ChildrenTotal как сумма Kidhome и Teenager:

In [22]:
df_sorted_col['ChildrenTotal'] = df_sorted_col['Kidhome'] + df_sorted_col['Teenhome']

Сохраню итоговый датафрейм

In [23]:
# df_sorted_col.to_csv('data/ifood_processed.csv')

## [Дашборд в Tableau](https://public.tableau.com/views/TestTask_16673711996610/Dashboard1?:language=en-US&publish=yes&:display_count=n&:origin=viz_share_link)

## Выводы:

* Конкретный Store посещают больше люди с высшим образованием. Чем образованнее человек, тем выше траты и средний чек. 
* Принятие рекламной кампании положительно коррелирует с доходом и отрицательно коррелирует с наличием детей
* Доход косвенно влияет на сумма расходов и имеет отрицательную корреляцию с количеством детей дома. Сумма, потраченная на вино, связана с потраченной суммой на мясо и его покупку в каталоге или в магазинах. Количество детей отрицательно связано с доходом, общей суммой расходов и, следовательно, с вином. Более высокий доход также связан с принятием кампаний
* Наиболее продаваемыми продуктами являются вина и мясо (средний покупатель тратит больше всего на эти товары). Можно подумать над созданием рекомендательной системы мяса к вину и наоборот


* Предложения:
    * Можно создать несколько потоков целевых рекламных кампаний:
        * Один из которых нацелен на людей с высоким доходом без детей/подростков. Акцент на элитарность алкоголя, мяса и рыбы. Тем более, что есть продукты категории Gold 
        * Другой – на людей с низким доходом, имеющих детей/подростков
    * Либо сосредоточить рекламные кампании на увеличении продаж менее популярных товаров
    * Сосредоточить рекламные кампании на более успешных каналах, чтобы охватить больше клиентов

  
    
* Неэффективные каналы совершения покупок — это покупки по каталогу и через дисконт. Наиболее эффективными каналами являются покупки в Интернете и в магазине (т. е. средний покупатель совершил больше всего покупок через эти каналы)
    * Лучше сосредоточить рекламные кампании на более успешных каналах, чтобы охватить больше клиентов